In [1]:
import sys
from datetime import datetime
from dateutil import parser
import pandas as pd
import numpy as np
import random
from IPython.display import HTML

import json 

%load_ext autoreload
%autoreload 2

# %logstart -t -o 'Logs/ken | create basic | .log'

pd.options.display.max_colwidth = 140
pd.options.display.mpl_style = 'default'

%matplotlib inline
import matplotlib.pyplot as plt

/Users/gauravparuthi/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2885: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
import common.mturk as mturk
from datetime import datetime
import pytz
import math
import common.config as config

def get_mturk_conf(use_sandbox=True):
    requester_conf = {
        "use_sandbox" : use_sandbox,
        "stdout_log" : False,
        "verify_mturk_ssl" : False,
        "aws_key" : config.AWS_KEY,
        "aws_secret_key" : config.AWS_SECRET_KEY
    }
    print datetime.now(), requester_conf
    return requester_conf

def get_balance(use_sandbox=False):
    requester_conf = get_mturk_conf(use_sandbox)
    r = mturk.MechanicalTurk(requester_conf).request("GetAccountBalance")
    if r.valid:
        return r.get_response_element("AvailableBalance")

def get_question(url):
    return """<?xml version="1.0" encoding="UTF-8"?>
        <ExternalQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07-14/ExternalQuestion.xsd">
          <ExternalURL>%s</ExternalURL>
          <FrameHeight>800</FrameHeight>
        </ExternalQuestion>
    """%url

no_of_days = 2
def create_hit(task_config):
    m = mturk.MechanicalTurk(get_mturk_conf(task_config['use_sandbox']))
    description = task_config['description']
    keywords = task_config['keywords']
    incentive = task_config['incentive']
    amt = math.ceil(incentive*100)/100
    reward = {'Amount':amt, 'CurrencyCode':'USD'}
    duration= 60*20
    autoapp = 60*60*48
    question= get_question(task_config['url'])
    lifetime = 60*60*24*no_of_days 
    max_assignments = task_config['max_assignments']
    qualification_req = [{"QualificationTypeId":"000000000000000000L0",
                                "IntegerValue":98,
                                'Comparator':'GreaterThan'},{"QualificationTypeId":"00000000000000000040",
                                "IntegerValue":1000 if task_config['use_sandbox']==False else 1,
                                'Comparator':'GreaterThan'},
                                {"QualificationTypeId":"00000000000000000071",
                                'Comparator':'EqualTo', 'LocaleValue':{'Country':'US'}},
                                ]

    task_type_conf = {'Title': task_config['title'], 
                               'Description' :description, 
                               'MaxAssignments': max_assignments,
                               'Keywords':keywords, 
                               'Reward':reward, 
                               'RequesterAnnotation':task_config['title'],
                               'AssignmentDurationInSeconds':duration, 
                               'AutoApprovalDelayInSeconds': autoapp,
                               'Question': question,
                               'LifetimeInSeconds': lifetime,
                               'QualificationRequirement': qualification_req
    }


    r = m.request("CreateHIT",task_type_conf)
    try: 
      print 'HITId: %s'%r['CreateHITResponse']['HIT']['HITId']
    except:
      print 'Some problem. AWS Response: %s'%r 
    
    return r

def extend_hit_assignments(hitid, count):
    m = mturk.MechanicalTurk(get_mturk_conf(False))
    r = m.request("ExtendHIT",
              {'HITId': hitid, 
               'MaxAssignmentsIncrement':count
               })
    print r
    return r

def get_assignments(hitid, pageno=1):
    m = mturk.MechanicalTurk(get_mturk_conf(False))
    r = m.request("GetAssignmentsForHIT",{'HITId':hitid,'PageSize':100, 'PageNumber': pageno})
    print 'total assignments = %s'%r['GetAssignmentsForHITResponse']['GetAssignmentsForHITResult']['TotalNumResults']
    print 'in this page = %s'% len(r['GetAssignmentsForHITResponse']['GetAssignmentsForHITResult']['Assignment'])
    return r

def reject_assignment(assignmentId, feedback='Sorry, You failed to rate correctly on our verification message. Please read the instructions carefully next time. More information can be found at https://goo.gl/QAkHaJ. Please contact the requester for more information.'):
    m = mturk.MechanicalTurk(get_mturk_conf(False))
    r = m.request("RejectAssignment",
              {
               'AssignmentId': assignmentId, 
               'RequesterFeedback': feedback
            })
    print r
    return r

def approve_rejected_assignment(assignmentId):
    m = mturk.MechanicalTurk(get_mturk_conf(False))
    r = m.request("ApproveRejectedAssignment",
              {
               'AssignmentId': assignmentId, 
               'RequesterFeedback': 'Approving some rejected assignments just this time.'
            })
    print r
    return r

def block_worker(workerId):
    if db.blocked_workers.find_one({'WorkerId':workerId}):
        return {'msg':'already blocked', 'workerId':workerId}
    m = mturk.MechanicalTurk(get_mturk_conf(False))
    q= {
            'WorkerId': workerId, 
            'Reason': 'Task was not completed properly. Rating of messages was found to be inappropriate: you failed to vote correctly on our verification messages.'
               }
    db.blocked_workers.insert(q)
    r = m.request("BlockWorker",
              q)
    print r
    return r


def gen_operation(op,q):
    # if db.unblocked_workers.find_one({'WorkerId':workerId}):
        # return {'msg':'already blocked', 'workerId':workerId}
    m = mturk.MechanicalTurk(get_mturk_conf(False))
    # q= {
    #         'WorkerId': workerId, 
    #         'Reason': 'Task was not completed properly. Rating of messages was found to be inappropriate: you failed to vote correctly on our verification messages.'
    #            }
    # db.insert(q)
    r = m.request(op,q)
    print r
    return r

def grant_bonus(assignment_id, amt, reason):
    worker_id = db.responses.find_one({'assignmentId':assignment_id}).get('workerId')
    print 'found workerId %s'%worker_id
    m = mturk.MechanicalTurk(get_mturk_conf(False))
    r = m.request("GrantBonus",
              {
               'AssignmentId': assignment_id, 
               'WorkerId':worker_id,
               'BonusAmount': {'Amount':amt, 'CurrencyCode':'USD'},
               'Reason':reason
            })
    return r

def gen_operation(op,q):
    m = mturk.MechanicalTurk(get_mturk_conf(False))
    r = m.request(op,q)
    return r

def disable_hit(hitid):
    return gen_operation('DisableHIT',{'HITId':hitid})

In [11]:
get_balance()

2016-06-15 17:28:33.197950 {'stdout_log': False, 'verify_mturk_ssl': False, 'use_sandbox': False}


/Users/gauravparuthi/miniconda2/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


{u'Amount': u'214.050', u'CurrencyCode': u'USD', u'FormattedPrice': u'$214.05'}

In [8]:
keywords = 'message, text, walking, support, help, physical, activity'

task_config = {'title':'Evaluate messages on their quality',
               'url':'https://turkeval.firebaseapp.com/',
              'description': 'Help someone to walk more by evaluating short messages that will be sent.',
              'keywords':'image, tag, age',
              'incentive': 1,
              'max_assignments':100,
              'use_sandbox': False} 

In [9]:
create_hit(task_config)

2016-06-15 11:29:23.580747 {'stdout_log': False, 'verify_mturk_ssl': False, 'use_sandbox': False}


/Users/gauravparuthi/miniconda2/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


HITId: 356ZPKYPUHTXCI3FORM6FN7WE33YP7


{u'CreateHITResponse': {u'HIT': {u'HITId': u'356ZPKYPUHTXCI3FORM6FN7WE33YP7',
   u'HITTypeId': u'307F0QTJA1FUZZFEGWCWAVNS1VGA8D',
   u'Request': {u'IsValid': u'True'}},
  u'OperationRequest': {u'RequestId': u'537b0d69-d5b1-4af8-a8f0-a4da3c289784'}}}

## Makeup HIT

In [ ]:
keywords = 'message, text, walking, support, help, physical, activity'

task_config = {'title':'[Exclusive HIT] Please Ignore',
               'url':'https://intecolab.com:5000/dummyIncentiveHit',
              'description': 'Only those people who have been contacted by the requester may submit.',
              'keywords':'image, tag, age',
              'incentive': 1,
              'max_assignments':9,
              'use_sandbox': True} 


In [ ]:
create_hit(task_config)

In [ ]:
findl(db.responses)

In [ ]:
hitids = ['A9KLWB70I12U7','A38ZV8LBZ034IZ','A5N6QK76RTR7R','A2CGCAQA2IKLRV','APEQ60QNQ3WQT','ADTNOFJHTTB1L','A2R1JVEVFXAGUI','A28L1K6D8QUCML','A9KLWB70I12U7','APEQ60QNQ3WQT']
[(x['workerId'], x['assignmentId']) for x in db.responses.find({'workerId': {'$in': hitids}})]

In [ ]:
grant_bonus('3BXQMRHWK0ACWQKRCC2RJI28ZT3UMR','1','for the bug')

## Operations on Assignments

In [ ]:
import xmltodict, json

def get_answer(ass):
    return tuple([xmltodict.parse(ass['Answer'])['QuestionFormAnswers']['Answer']['FreeText'],])

In [ ]:
for rec in r[u'GetAssignmentsForHITResponse'][u'GetAssignmentsForHITResult']['Assignment']:
    oidct =  xmltodict.parse(rec['Answer'])[u'QuestionFormAnswers']['Answer']

In [ ]:
response = pd.DataFrame(oidct).set_index('QuestionIdentifier').to_dict()['FreeText']#orient='records')

In [ ]:
data = {}

def checkmid(mid):
    if mid not in data:
        data[mid] = {}


for k in response:
    if 'message' in k and 'messagee' not in k:
        print k
        if '_text' in k:
            text = response[k]
            mid = k[7:9]
            checkmid(mid)
            data[mid]['text'] = text
        

data

In [ ]:
extend_hit_assignments('3K3G488TR2KKA6UAD2WY4VZDL8SQ56',10)

In [ ]:
assignmentids = ['3JC6VJ2SACVU9ALNE84E2FFS6VM5A1',
    '3JBT3HLQF9EZ3SBEAFTRHEBI6GTZPH',
    '3LUY3GC630CGJI58RK18QUIJWNH7PY',
    '39U1BHVTDM3WARXS6P72IN70GOGT3K',
    '30LB5CDZNDMWFB7UTU9JXHL6HNF0ZN']
for aid in assignmentids:
    reject_assignment(aid, feedback="It has been noted that you skipped rating some of the messages. You might have clicked the Next button without completing the ratings. Please contact the requester if you have any questions.")

#### Disable HIT

In [ ]:
disable_hit('3OQQD2WO8II4FTJNMB3WGWA7G193ID')

# HTML must contain

A form that submits to Turk. Example: 

```
<form id="turkform" action={getUrlParameter("turkSubmitTo")+"/mturk/externalSubmit"} method="post">
```

Where getUrlParameter is a function that gets the value of a parameter from URL. "turkSubmitTo" is passed by MTurk to the page. 